In [ ]:
import timeit
import requests
import json
import pandas as pd
from io import BytesIO, StringIO
import numpy as np
from datetime import datetime
import oauth2client
from oauth2client.client import GoogleCredentials
from oauth2client import client
from oauth2client.client import flow_from_clientsecrets
from oauth2client import file
from oauth2client import tools
from googleapiclient import discovery
from googleapiclient import errors
from google_auth_oauthlib import flow
import google_auth_httplib2
import httplib2
import urllib.request
import time
import os
import ast
import io
from io import StringIO
from googleapiclient.http import MediaIoBaseDownload
from venv import create
import googleapiclient.discovery
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import json
#!/usr/bin/env python

import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
df1 = pd.read_csv('/Users/hamzaahmed/Coding Projects/Test/playground/Assignments Import - Assignments.csv')
df2 = pd.read_csv('/Users/hamzaahmed/Coding Projects/Test/playground/Legacy Sheet (GCP) - Table.csv')

In [ ]:
# Concatenate the DataFrames vertically
concatenated = pd.concat([df1, df2])

# Drop the duplicate rows keeping the first occurrence
deduplicated = concatenated.drop_duplicates(keep='first')

# Filter the rows that are only in df1
missing_rows = deduplicated.loc[deduplicated.index.isin(df1.index) & ~deduplicated.index.isin(df2.index)]

# missing_rows.to_csv('missing.csv')

In [ ]:
# Convert boolean columns to string
boolean_cols = missing_rows.select_dtypes(include=bool).columns
missing_rows[boolean_cols] = missing_rows[boolean_cols].astype(str)

In [ ]:
OAUTH_SCOPES = ("https://www.googleapis.com/auth/drive " + "https://www.googleapis.com/auth/drive.file " + "https://www.googleapis.com/auth/spreadsheets ")

In [ ]:
flow = client.flow_from_clientsecrets(
    "/Users/hamzaahmed/Coding Projects/Test/playground/Google Sheet Access/Google Sheet Creds.json", scope = OAUTH_SCOPES, 
    redirect_uri = "urn:ietf:wg:oauth:2.0:oob")

storage = oauth2client.file.Storage('/Users/hamzaahmed/Coding Projects/Test/playground/Google Sheet Access/Credentials.dat')
credentials = storage.get()

if credentials is None or credentials.invalid:
  credentials = tools.run_flow(flow, storage,
                               tools.argparser.parse_known_args()[0])
http = credentials.authorize(httplib2.Http())


service = discovery.build('sheets', 'v4', http=http)

In [ ]:
SPREADSHEET_ID=''
SHEET_NAME = "Sheet1"
range = '{0}!A1:ZZ'.format( SHEET_NAME )
clear_values_request_body = {
    # TODO: Add desired entries to the request body.
}

request_clear = service.spreadsheets().values().clear(spreadsheetId=SPREADSHEET_ID, range=range, body=clear_values_request_body)
response = request_clear.execute()


In [ ]:
# Convert NaN values to None
df23 = df2.where(pd.notnull(df2), None)

In [ ]:
df2 = df2.fillna(" ")
values = [df2.columns.values.tolist()] + df2.values.tolist()
data = [
    {
        'range': f"'{SHEET_NAME}'!A1",
        'values': values
    },
    # Additional ranges to update ...
]
body = {
    'valueInputOption': "RAW",
    'data': data

}

In [ ]:
result = service.spreadsheets().values().batchUpdate(
    spreadsheetId=SPREADSHEET_ID, body=body).execute()
print('{0} cells updated.'.format(result.get('totalUpdatedCells')))